This notebook will look at modelling the affect of gravity on falling objects, by producing data mimicing these objects and then create machine learning models that learn from this data.

## Modelling $y = x_0 + v_0t - \frac{1}{2}gt^2$

First, we create a function to calculate the position of a object after dropping it. This is found with the above formula. Note that,

$x_0 =$ initial position

$v_0 =$ initial velocty

$g =~ 9.8 m/s^2$ (approximately)

In [1]:
def location(x_0, v_0, t):
    return x_0 + v_0*t - (9.8/2)*t**2

Now we will create a dataset, based on random input values, with the location of the object calculated from the function above. Note that we will include the mass as a variable, even though it is irrelevant to the resulting location. We will see how our machine learning algorithm handles this.

In [2]:
import csv
import random
import math
random.seed

with open('gravity_location_data.csv', mode='w') as gravity_file:
    gravity_writer = csv.writer(gravity_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    gravity_writer.writerow(['initial_position', 'initial_velocity', 'mass', 'time', 'location'])

    for i in range (0, 10000):
        initial_position = random.randrange(1, 10000)
        initial_velocity = random.randrange(1, 100)
        mass = random.randrange(1, 1000)
        time = random.randrange(1, 100)

        gravity_writer.writerow([initial_position, initial_velocity, mass, time, location(initial_position, initial_velocity, time)])


Next, we get this dataset into our application.

In [3]:
import pandas as pd

gravity_data = pd.read_csv('gravity_location_data.csv')
df_location = pd.DataFrame(gravity_data)

We create a function to split the data into training and testing samples.

In [4]:
from sklearn.model_selection import train_test_split

def split_data(data, target_name):
    y = data[target_name]
    X = data.drop(target_name, axis=1)
    return train_test_split(X, y, test_size=0.2, random_state=30)

Then we create a function that can accept our train and test data, train a model on this data, and evaluate it.

In [5]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt

def train_eval_poly(X_train, X_test, y_train, y_test):
    regression_model = LinearRegression() 
    
    poly = PolynomialFeatures(degree=2)
    X_train_transform = poly.fit_transform(X_train)
    X_test_transform = poly.fit_transform(X_test)
    
    regression_model.fit(X_train_transform, y_train)
    
    print(poly.fit(X_train).get_feature_names(X_train.columns))
    
    y_pred = regression_model.predict(X_test_transform)
    print("R2: \t", r2_score(y_test, y_pred))
    print("RMSE: \t", sqrt(mean_squared_error(y_test, y_pred)))
    print("MAE: \t", mean_absolute_error(y_test, y_pred))
    
    return regression_model 

Now we use our functions to train our model.

In [6]:
df_split = split_data(df_location, 'location')
lrModel = train_eval_poly(*df_split)

['1', 'initial_position', 'initial_velocity', 'mass', 'time', 'initial_position^2', 'initial_position initial_velocity', 'initial_position mass', 'initial_position time', 'initial_velocity^2', 'initial_velocity mass', 'initial_velocity time', 'mass^2', 'mass time', 'time^2']
R2: 	 1.0
RMSE: 	 2.999978554169189e-09
MAE: 	 2.4458151883788302e-09


The evaluation shows very good results, particularly the $r^2$ of 1.0! Now let's look at the coefficients produced on the model.

In [7]:
lrModel.coef_

array([ 0.00000000e+00,  1.00000000e+00, -6.75868639e-13, -2.45002508e-14,
        4.20460132e-13,  2.39542363e-16,  1.80762368e-17, -2.55803435e-17,
        4.07539983e-16, -9.27300961e-15, -6.81456968e-16,  1.00000000e+00,
        4.59909681e-16, -1.63791735e-15, -4.90000000e+00])

These coefficients match the output of the bracketed array in the previous results. Note most of these values are extermely small--they are essentially zero. This includes the value for mass, because the training operation was able to determine, more or less, that this value is irrelevant. The final value is the gravity constant, or more specifically, $-g/2$ or $-9.8/2$. Finally, the second and twelfth values are 1. These correspond to initial_position initial_velocity*time.

In other words, this set of coefficients show that we have trained a model that corresponds to the equation of a falling object.

Next, we will look at a slightly harder problem, which is, predicting the time it will take an object to hit the ground, falling from a certain height at a certain initial velocity. First, we calculate the "fall time", which is the same as our equation for a falling object, but we solve for t. This gives us...

## $t = -v_0 - \frac{\sqrt{v_0^2 + 19.2x_0}}{-9.8}$

Now we create a function that will calculate the fall time of an object.

In [8]:
def fall_time(initial_position, initial_velocity):
    a = -9.8 / 2
    b = initial_velocity
    c = initial_position
    
    time = (-1 * b - math.sqrt(b**2 + 19.6*c)) / -9.8
    return time



Then we create a dataset of randomized values, with their resulting fall time.

In [9]:
with open('gravity_fall_data.csv', mode='w') as gravity_file:
    gravity_writer = csv.writer(gravity_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    gravity_writer.writerow(['initial_position', 'initial_velocity', 'mass', 'fall_time'])

    for i in range (0, 10000):
        initial_position = random.randrange(1, 10000)
        initial_velocity = random.randrange(1, 1000)
        mass = random.randrange(1, 1000)

        gravity_writer.writerow([initial_position, initial_velocity, mass, fall_time(initial_position, initial_velocity)])
              

And input it.

In [10]:
gravity_data = pd.read_csv('gravity_fall_data.csv')
df_fall = pd.DataFrame(gravity_data) 

Now let's train our model based on this data. First we will use strict linear regression.

In [11]:
def train_eval_linear(X_train, X_test, y_train, y_test):
    regression_model = LinearRegression()
    regression_model.fit(X_train, y_train)
    y_pred = regression_model.predict(X_test)
    print("R2: \t", r2_score(y_test, y_pred))
    print("RMSE: \t", sqrt(mean_squared_error(y_test, y_pred)))
    print("MAE: \t", mean_absolute_error(y_test, y_pred))
    return regression_model
    
df_split = split_data(df_fall, 'fall_time')
lrModel = train_eval_linear(*df_split)

R2: 	 0.995527175268824
RMSE: 	 3.551847761580043
MAE: 	 2.752993296307049


In [12]:
lrModel.coef_

array([1.94907261e-03, 1.83824083e-01, 4.37656545e-05])

This is quite good! Suprisingly so. Now let's try polynomial regresion.

In [13]:
df_split = split_data(df_fall,'fall_time')
lrModel = train_eval_poly(*df_split)

['1', 'initial_position', 'initial_velocity', 'mass', 'initial_position^2', 'initial_position initial_velocity', 'initial_position mass', 'initial_velocity^2', 'initial_velocity mass', 'mass^2']
R2: 	 0.9994631902517299
RMSE: 	 1.2304772811300295
MAE: 	 0.9315999956362887


This is even better! There was not much room for improvement, but we found it.

In [14]:
lrModel.coef_

array([ 0.00000000e+00,  3.91131574e-03,  1.65739891e-01, -1.46354722e-04,
       -5.85570688e-08, -2.74996796e-06,  2.79424189e-09,  3.16862471e-05,
        2.24127968e-08,  1.15017972e-07])

Notice, however, that in both cases the coefficients are pretty indecipherable, or at least, they cannot obviously be transformed back to our original equation. That's fine. Machine learning is not necessarily the best way to find the correct equation, but in some cases it does quite well.